<a href="https://colab.research.google.com/github/miaomiao612/MiniGPT-4/blob/main/finetune_miniGPT4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/miaomiao612/MiniGPT-4.git

Cloning into 'MiniGPT-4'...
remote: Enumerating objects: 419, done.
remote: Counting objects: 100% (280/280), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 419 (delta 124), reused 213 (delta 104), pack-reused 139
Receiving objects: 100% (419/419), 91.31 MiB | 22.93 MiB/s, done.
Resolving deltas: 100% (141/141), done.


In [2]:
%cd /content/MiniGPT-4

/content/MiniGPT-4


# **1.2 准备Vicuna权重**

### 1. Vicuna weights:

In [3]:
!git lfs install

!git clone https://huggingface.co/lmsys/vicuna-7b-delta-v0  # smaller, need 12G gpu memory

Updated git hooks.
Git LFS initialized.
Cloning into 'vicuna-7b-delta-v0'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 32 (delta 2), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (32/32), 7.11 KiB | 1.02 MiB/s, done.
Filtering content: 100% (3/3), 4.55 GiB | 40.30 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


### 2. LLAMA-7B weight:

In [5]:
!git clone https://huggingface.co/decapoda-research/llama-7b-hf

Cloning into 'llama-7b-hf'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), 22.83 KiB | 1.20 MiB/s, done.
Filtering content: 100% (34/34), 12.55 GiB | 150.83 MiB/s, done.


### 3. 下载兼容library

In [4]:
!pip install git+https://github.com/lm-sys/FastChat.git@v0.1.10

  Cloning https://github.com/lm-sys/FastChat.git (to revision v0.1.10) to /tmp/pip-req-build-8jnghetf
  Running command git clone --filter=blob:none --quiet https://github.com/lm-sys/FastChat.git /tmp/pip-req-build-8jnghetf
  Running command git checkout -q f34f28cedcb8906fd026f22ec3ef41435a8e24ac
  Resolved https://github.com/lm-sys/FastChat.git to commit f34f28cedcb8906fd026f22ec3ef41435a8e24ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-h3ffb1wx/transformers_49c0075778d24b4ca96eee5ed6130408
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-h3ffb1wx/transformers_49c0075778d24b4ca96eee5ed6130408
  Resolved https://github.com/huggingface/transformers.git to commit 9e28750287df57942d716083ae53bb4e766104c2
  In

### 4. 创建最终权重

将llama-7b-hf/tokenizer_config.json 中的"tokenizer_class": “LLaMATokenizer” 改成 “tokenizer_class”: “LlamaTokenizer”

In [6]:
!python -m fastchat.model.apply_delta --base /content/MiniGPT-4/llama-7b-hf/  --target /content/MiniGPT-4/vicuna_weights/  --delta /content/MiniGPT-4/vicuna-7b-delta-v0/

Loading the base model from /content/MiniGPT-4/llama-7b-hf/
2023-06-29 19:44:16.194232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100% 33/33 [00:17<00:00,  1.89it/s]
Loading the delta from /content/MiniGPT-4/vicuna-7b-delta-v0/
Loading checkpoint shards: 100% 2/2 [00:54<00:00, 27.46s/it]
Applying the delta
Applying delta: 100% 323/323 [00:06<00:00, 48.34it/s]
Saving the target model to /content/MiniGPT-4/vicuna_weights/


修改minigpt4/configs/models/minigpt4.yaml中“llama_model”为vicuna_weights路径

# 1.3 微调

pretrained MiniGPT-4 checkpoint

In [7]:
!wget https://huggingface.co/wangrongsheng/MiniGPT4-7B/resolve/main/prerained_minigpt4_7b.pth


--2023-06-29 19:46:58--  https://huggingface.co/wangrongsheng/MiniGPT4-7B/resolve/main/prerained_minigpt4_7b.pth
Resolving huggingface.co (huggingface.co)... 65.9.86.57, 65.9.86.79, 65.9.86.62, ...
Connecting to huggingface.co (huggingface.co)|65.9.86.57|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/25/fa/25fa880e116eda3d82c5c87b32eb0a7fa8f76b139d70ce756851215ce7a76179/017a9ed588a11ed383711003cf50cf675191420a04689f682fb56fa9bbb8dcbb?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27prerained_minigpt4_7b.pth%3B+filename%3D%22prerained_minigpt4_7b.pth%22%3B&Expires=1688327218&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzI1L2ZhLzI1ZmE4ODBlMTE2ZWRhM2Q4MmM1Yzg3YjMyZWIwYTdmYThmNzZiMTM5ZDcwY2U3NTY4NTEyMTVjZTdhNzYxNzkvMDE3YTllZDU4OGExMWVkMzgzNzExMDAzY2Y1MGNmNjc1MTkxNDIwYTA0Njg5ZjY4MmZiNTZmYTliYmI4ZGNiYj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOn

在train_configs stage2中修改MiniGPT-4 checkpoint路径

在minigpt4/configs/datasets/cc_sbu/align.yaml修改为数据集路径

In [8]:
!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [81.0 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,07

In [9]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 50.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 29.3 MB/s et

将下面代码中train.py前的数字改为自己GPU的数量， 这里为1

### 模型训练

In [10]:
!torchrun --nproc-per-node 1 train.py --cfg-path train_configs/minigpt4_stage2_finetune.yaml

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
2023-06-29 20:01:37.349446: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
| distributed init (rank 0, world 1): env://
2023-06-29 20:01:39,089 [INFO] 
=====  Running Parameters    =====
2023-06-29 20:01:39,089 [INFO] {
    "amp": true,
    "batch_size_eval": 12,
    "batch_size_train": 12,
    "device": "cuda",
    "dist_backend": "nccl",
    "dist_url": "env://",
    "distributed": true,
    "evaluate": false,
    "gpu": 0,
    "init_lr": 3e-05,
    "iters_per_epoch": 200,
    "lr_sched": "linear_warmup_cosine_lr",
    "max_epoch": 5,
    "min_lr": 1e-05,
    "num_workers": 4,
    "output_dir": "output/minigpt4_stage2_finetune",
    "rank": 0,
    "r

#1.4 模型评估

eval_configs/minigpt4_eval.yaml中的第 11 行设置为待评估模型的checkpoint路径

In [ ]:
!python demo.py --cfg-path eval_configs/minigpt4_eval.yaml  --gpu-id 0

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
2023-06-29 02:34:29.908276: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing Chat
Loading VIT
Loading VIT Done
Loading Q-Former
Loading Q-Former Done
Loading LLAMA

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
Loading checkpoint shards: 100% 2/2 [00:13<00:00,  6.77s/it]
Loading LLAMA Done
Load 4 training prompts
Prompt Example 
###Human: <Img><ImageHere></Img> Please provide a detailed description of the picture. ###Assistant: 
Load BLIP2-LLM Checkpoint: /content/MiniGPT-4/minigpt

# 微调前：

In [ ]:
!python demo.py --cfg-path eval_configs/minigpt4_eval.yaml  --gpu-id 0

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
2023-06-28 20:37:28.002628: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing Chat
Loading VIT
Loading VIT Done
Loading Q-Former
Loading Q-Former Done
Loading LLAMA

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
Loading checkpoint shards: 100% 2/2 [00:13<00:00,  6.77s/it]
Loading LLAMA Done
Load 4 training prompts
Prompt Example 
###Human: <Img><ImageHere></Img> Take a look at this image and describe what you notice. ###Assistant: 
Load BLIP2-LLM Checkpoint: /content/MiniGPT-4/prera